# Testing C++ code in python
June 12, 2020 \
June 20, 2020 : Added test for batch of images with multi-channels

In [1]:
import numpy as np

In [2]:
xsize,ysize=128,128
# xsize,ysize=10,10
batch_size,num_channels=100,2

## Save image for C++ code

In [ ]:
# # ### Save single 2D image to csv for C++ code to handle
# fname='data/gen_images.npy'
# a1=np.load(fname)
# print(a1.shape)

# a2=a1[0:batch_size*num_channels,0,:xsize,:ysize]
# print(a2.shape)
# np.savetxt('data/images.csv',a2.flatten(),delimiter=',',newline=',')

# ### Save second image
# a2=a1[1*batch_size*num_channels:2*batch_size*num_channels,0,:xsize,:ysize]
# print(a2.shape)
# np.savetxt('data/images2.csv',a2.flatten(),delimiter=',',newline=',')

### Test for single image

In [ ]:
# fname='data/images.csv'
# x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(xsize,ysize)
# print(x.shape)
# # x


# arr=np.array([(i+1)*10+ 5*j for i in range(0,5) for j in range (0,5)]).reshape(5,5).T
# arr


# y1=np.fft.fft2(arr)
# # y1=np.fft.fftshift(y1)
# y2=abs(y1)

In [ ]:
# ### Read c++ file
# cp_ip_file='data/op.csv'
# # x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(5,5)
# z_cpp=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64)
# print(z_cpp)

In [ ]:
# ### Check arrays
# np.allclose(z1,z_cpp[:z1.shape[0]],rtol=1e-4,atol=1e-8)

In [ ]:
# ### Check manually elementwise
# for i in range(z1.shape[0]):
#     if (np.around(z1[i],4)!=np.around(z_cpp[i],4)): 
#           print(i,z1[i],z_cpp[i])

## Multi-channel batch example

### Read input

In [28]:
fname='data/images.csv'
x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(batch_size,num_channels,xsize,ysize)
print(x.shape)
# x

(100, 2, 128, 128)


### Compute batch spectrum

In [29]:
def f_radial_profile(data, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    if center[0]==None and center[1]==None:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0]) # compute centers
        
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    
    return radialprofile

def f_compute_spectrum(arr):
#     GLOBAL_MEAN=1.0
#     arr=((arr - GLOBAL_MEAN)/GLOBAL_MEAN)
    y1=np.fft.fft2(arr)
    y1=np.fft.fftshift(y1)
    y2=abs(y1)**2
    z1=f_radial_profile(y2)
    return(z1)
   
def f_compute_batch_spectrum(arr):
    batch_pk=np.array([f_compute_spectrum(i) for i in arr])
    return batch_pk


### Code ###
def f_image_spectrum(x):
    '''
    Data has to be in the form (batch,channel,x,y)
    '''
    print(x.shape)
    mean=[[] for i in range(num_channels)]    
    sdev=[[] for i in range(num_channels)]    

    for i in range(num_channels):
        arr=x[:,i,:,:]
#         print(i,arr.shape)
        batch_pk=f_compute_batch_spectrum(arr)
#         print(batch_pk)
        mean[i]=np.mean(batch_pk,axis=0)
        sdev[i]=np.std(batch_pk,axis=0)
    mean=np.array(mean)
    sdev=np.array(sdev)
    return mean,sdev


In [30]:
mean,sdev=f_image_spectrum(x)
print(mean.shape,sdev.shape)

(100, 2, 128, 128)
(2, 90) (2, 90)


### Read from c++ files and compare

In [31]:
### Read c++ files
cp_ip_file='data/op_spec_mean.csv'
# x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(5,5)
z_cpp1=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(num_channels,-1)

cp_ip_file='data/op_spec_sdev.csv'
# x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(5,5)
z_cpp2=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(num_channels,-1)
# print(z_cpp1,'\n',z_cpp2)

In [32]:
z_cpp1

array([[3.35616e+07, 8.37788e+04, 6.53382e+04, 4.53748e+04, 3.71107e+04,
        2.99426e+04, 2.47575e+04, 2.04060e+04, 1.75443e+04, 1.41365e+04,
        1.24007e+04, 1.10736e+04, 9.41233e+03, 8.11342e+03, 6.94294e+03,
        6.23430e+03, 5.45892e+03, 5.01255e+03, 4.10544e+03, 3.45309e+03,
        3.01188e+03, 2.63551e+03, 2.38433e+03, 2.13505e+03, 1.91546e+03,
        1.68012e+03, 1.47601e+03, 1.37052e+03, 1.31731e+03, 1.14710e+03,
        1.00499e+03, 1.00979e+03, 9.47904e+02, 7.80218e+02, 6.87833e+02,
        6.48472e+02, 5.81977e+02, 5.06747e+02, 4.62655e+02, 4.28049e+02,
        4.18774e+02, 3.81848e+02, 3.40941e+02, 3.12729e+02, 2.81141e+02,
        2.93893e+02, 2.47112e+02, 2.52697e+02, 2.47536e+02, 2.17739e+02,
        1.85645e+02, 1.88557e+02, 1.68892e+02, 1.60808e+02, 1.54097e+02,
        1.49148e+02, 1.55979e+02, 1.54880e+02, 1.41140e+02, 1.27993e+02,
        1.23136e+02, 1.20102e+02, 1.11583e+02, 1.17513e+02, 1.02039e+02,
        8.48641e+01, 7.10699e+01, 6.99338e+01, 7.30

In [33]:
mean

array([[3.35615801e+07, 8.37787838e+04, 6.53382318e+04, 4.53748105e+04,
        3.71107162e+04, 2.99426116e+04, 2.47574663e+04, 2.04059979e+04,
        1.75442856e+04, 1.41365088e+04, 1.24006927e+04, 1.10735809e+04,
        9.41232667e+03, 8.11342311e+03, 6.94294066e+03, 6.23430462e+03,
        5.45891791e+03, 5.01254670e+03, 4.10543565e+03, 3.45308767e+03,
        3.01188035e+03, 2.63551069e+03, 2.38433352e+03, 2.13504891e+03,
        1.91545771e+03, 1.68012075e+03, 1.47601015e+03, 1.37052451e+03,
        1.31730895e+03, 1.14710158e+03, 1.00498640e+03, 1.00978722e+03,
        9.47904106e+02, 7.80217781e+02, 6.87832702e+02, 6.48472135e+02,
        5.81977342e+02, 5.06746732e+02, 4.62655301e+02, 4.28048814e+02,
        4.18773617e+02, 3.81848240e+02, 3.40941366e+02, 3.12729425e+02,
        2.81141319e+02, 2.93893096e+02, 2.47111890e+02, 2.52697377e+02,
        2.47535659e+02, 2.17739400e+02, 1.85644588e+02, 1.88556984e+02,
        1.68892349e+02, 1.60808213e+02, 1.54097189e+02, 1.491483

In [34]:
### Check arrays
print(np.allclose(mean,z_cpp1[:,:mean.shape[1]],rtol=1e-4,atol=1e-8))
print(np.allclose(sdev,z_cpp2[:,:sdev.shape[1]],rtol=1e-4,atol=1e-8))


True
True


## Conclusion:
###### July 2, 2020
Reading input + computing FFT + computing Modulus + Getting Radial profile \
Arrays from python and C++ match !!\
The C++ code is correct and ready to port to LBANN

###### July 20, 2020
Reading input + computing FFT + computing Modulus + Getting Radial profile + computing mean, sdev \
Arrays from python and C++ match !!\


####  Update
October 26, 2020 \
Adding a fix for $ fftshift $ \
Checked results for 2 different images. Success!!\

#### Update
Oct 30, 2020 \
Added square for abs value of spectrum \
Results match 

#### Update 
Nov 12, 2020 \
Note : both these codes compute the spectral err as sqrt{variance}. The pytorch code used to compute it as standard error.
Peter Harrington's code computes variance.

## Testing spectral loss

In [10]:
fname='data/images.csv'
x1=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(batch_size,num_channels,xsize,ysize)
fname='data/images2.csv'
x2=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(batch_size,num_channels,xsize,ysize)


In [11]:
mean1,sdev1=f_image_spectrum(x1)
mean2,sdev2=f_image_spectrum(x2)
print(mean1.shape)

(100, 2, 128, 128)
(100, 2, 128, 128)
(2, 90)


In [12]:
k_crop=int(xsize/2) ## =64
np.log(np.mean(np.square(mean1[:,:k_crop]-mean2[:,:k_crop]))),np.log(np.mean(np.square(sdev1[:,:k_crop]-sdev2[:,:k_crop])))

(22.307540969987837, 20.199064649307886)

### Conclusion:
July 28,2020

Results matches c++ code output exactly

#### Update 
October 26, 2020
Results matches after the fix for fftshift

#### Update
Oct 30, 2020
Added change to get square of spectrum.
Results match